# Step Detection

## Loading data

In [3]:
import sys

from datahandler.data_loader import load_data_from_file, load_date_from_steptracking_file
from datahandler.constants import v4_walking, location_labels, v4_mix
import os
from utils import print_line_divider
from datetime import timedelta
import pandas as pd
import numpy as np
from random import random, seed
from sklearn.preprocessing import MinMaxScaler

seed(100)

#### Load file names

In [4]:
# Loading file names
data_folder = v4_mix


def get_all_files_from_folder(folder):
    file_names = []
    for datafile in os.listdir(folder):
        if datafile.startswith("."):
            continue

        path = os.path.join(folder, datafile)
        if os.path.isfile(path):
            file_names.append(datafile)
    return file_names


data_dict = {}
for filename in get_all_files_from_folder(data_folder):
    code = filename[:3]
    data_dict[code] = os.path.join(data_folder, filename)

step_dict = {}
for filename in get_all_files_from_folder(data_folder + "/step_fixed"):
    code = filename[:3]
    if code in data_dict.keys():
        step_dict[code] = os.path.join(data_folder, "step", filename)

print(step_dict.keys())

dict_keys(['ds5', 'mm3', 'pp1', 'tt3', 'ds4', 'pp5', 'ds3', 'mm5', 'mm4', 'tt1', 'pp3', 'tt5'])


In [5]:
# Configuration
WINDOW_SIZE = 40
WINDOW_LENGTH_IN_SECONDS = 2

#### Load data from one file pair

In [6]:
# # Step 1 - Load data file
# code = "op1"
# df = load_data_from_file(data_dict[code])
# df = df.drop("labelActivity", axis=1)
# df['labelPhone'] = df['labelPhone'].apply(lambda x: location_labels.index(x))
# df

In [7]:
# # Step 2 - Load data file
# step_dates = load_date_from_steptracking_file(step_dict[code])
# print("There are " + str(len(step_dates)) + " steps found in this file")
# print(step_dates[int(len(step_dates) / 2)])

In [8]:
# # Step 3 - Divide collected data into fixed-size chunk
# fixed_size_data = []
# fixed_size_indexes = []
# current_timestamp = df.index[0].to_pydatetime()
# last_timestamp_raw = df.index[df.shape[0] - 1].to_pydatetime()
# current_timestamp_raw_index = 0
# one_window_length_in_millis = WINDOW_LENGTH_IN_SECONDS * 1000 / WINDOW_SIZE
# while True:
#     current_timestamp = current_timestamp + timedelta(milliseconds=one_window_length_in_millis)
#     if current_timestamp > last_timestamp_raw:
#         break
#
#     while current_timestamp_raw_index < df.shape[0] - 1:
#         next_timestamp_raw = df.index[current_timestamp_raw_index + 1].to_pydatetime()
#         if next_timestamp_raw < current_timestamp:
#             current_timestamp_raw_index += 1
#         else:
#             break
#
#     fixed_size_data.append(df.iloc[current_timestamp_raw_index])
#     fixed_size_indexes.append(current_timestamp)
# fixed_size_df = pd.DataFrame(
#     data=fixed_size_data,
#     index=fixed_size_indexes,
#     columns=df.columns
# )
#
# fixed_size_df

In [7]:
# # Step 4 - Add step date to this fixed_size_df
# def update_date_microsecond(date, first_rem, second_rem):
#     current_microsecond = date.microsecond
#     new_microsecond_one = (int(current_microsecond / 100000) * 100 + first_rem) * 1000
#     new_microsecond_two = (int(current_microsecond / 100000) * 100 + second_rem) * 1000
#     diff_one = abs(current_microsecond - new_microsecond_one)
#     diff_two = abs(current_microsecond - new_microsecond_one)
#     if diff_one > diff_two:
#         new_microsecond = new_microsecond_two
#     else:
#         new_microsecond = new_microsecond_one
#     return date.replace(microsecond=new_microsecond)
#
#
# index_first_date = fixed_size_df.index[0]
# print(type(index_first_date))
# milli = index_first_date.microsecond / 1000
# first_remainder = int(milli % 100)
# second_remainder = int((milli + 50) % 100)
#
# updated_step_dates = set()
# for date in step_dates:
#     updated_step_dates.add(update_date_microsecond(date, first_remainder, second_remainder))
#
# is_step_series = []
# count = 0
# for m in fixed_size_df.index:
#     is_step = m.to_pydatetime() in updated_step_dates
#     if is_step:
#         count += 1
#     is_step_series.append(is_step)
#
# print("Found " + str(count) + " steps in this data frame.")
#
# fixed_size_df['isStep'] = is_step_series
# fixed_size_df

In [8]:
# # Step 5 - Normalize the data
#
# def normalize(df):
#     feature_count = df.shape[1] - 2
#     values = df.iloc[:, 0:feature_count]
#     phone_labels = df.loc[:, "labelPhone"].set_axis(range(df.shape[0]))
#     is_step = df.loc[:, "isStep"].set_axis(range(df.shape[0]))
#     normalizer = MinMaxScaler()
#     normalized_values = normalizer.fit_transform(values)
#     normalized_df = pd.DataFrame(
#         data=normalized_values,
#         columns=values.columns
#     )
#     normalized_df["labelPhone"] = phone_labels
#     normalized_df["isStep"] = is_step
#     return normalized_df
#
#
# normalized_df = normalize(fixed_size_df)
# normalized_df

In [9]:
# # Step 6 - Slice the window
#
# def convert_df_to_final_train_data(df):
#     train_x, test_x = [], []
#     train_x_lite, test_x_lite = [], []
#     train_y_context, test_y_context = [], []
#     train_y_step, test_y_step = [], []
#     window_index_start = 0
#     window_index_increasing_size = 2
#     no_features = df.shape[1] - 2
#     values = df.iloc[:, 0:no_features]
#     context_labels = df.loc[:, "labelPhone"]
#     step_found = df.loc[:, "isStep"]
#
#     while window_index_start + WINDOW_SIZE < df.shape[0]:
#         train_data = values[window_index_start:(window_index_start + WINDOW_SIZE)]
#         train_data_lite = train_data[-2:]
#         context_label = context_labels[window_index_start]
#         is_step_list = step_found[window_index_start:(window_index_start + WINDOW_SIZE)].to_list()
#         is_step = is_step_list[-1] or is_step_list[-2]
#
#         if random() < 0.15:
#             test_x_lite.append(train_data_lite)
#             test_x.append(train_data)
#             test_y_context.append(context_label)
#             test_y_step.append(is_step)
#         else:
#             train_x_lite.append(train_data_lite)
#             train_x.append(train_data)
#             train_y_context.append(context_label)
#             train_y_step.append(is_step)
#
#         window_index_start += window_index_increasing_size
#
#     return np.array(train_x), np.array(train_x_lite), np.array(train_y_step), np.array(train_y_context), np.array(
#         test_x), np.array(test_x_lite), np.array(test_y_step), np.array(test_y_context)


## Building model & configuration

In [9]:
def load_data_from_code(code):
    # Step 1
    df = load_data_from_file(data_dict[code])
    df = df.drop("labelActivity", axis=1)
    df['labelPhone'] = df['labelPhone'].apply(lambda x: location_labels.index(x))
    # Step 2
    step_dates = load_date_from_steptracking_file(step_dict[code])
    # Step 3
    fixed_size_data = []
    fixed_size_indexes = []
    current_timestamp = df.index[0].to_pydatetime()
    last_timestamp_raw = df.index[df.shape[0] - 1].to_pydatetime()
    current_timestamp_raw_index = 0
    one_window_length_in_millis = WINDOW_LENGTH_IN_SECONDS * 1000 / WINDOW_SIZE
    while True:
        current_timestamp = current_timestamp + timedelta(milliseconds=one_window_length_in_millis)
        if current_timestamp > last_timestamp_raw:
            break

        while current_timestamp_raw_index < df.shape[0] - 1:
            next_timestamp_raw = df.index[current_timestamp_raw_index + 1].to_pydatetime()
            if next_timestamp_raw < current_timestamp:
                current_timestamp_raw_index += 1
            else:
                break

        fixed_size_data.append(df.iloc[current_timestamp_raw_index])
        fixed_size_indexes.append(current_timestamp)
    fixed_size_df = pd.DataFrame(
        data=fixed_size_data,
        index=fixed_size_indexes,
        columns=df.columns
    )

    # Step 4
    def update_date_microsecond(date, first_rem, second_rem):
        current_microsecond = date.microsecond
        new_microsecond_one = (int(current_microsecond / 100000) * 100 + first_rem) * 1000
        new_microsecond_two = (int(current_microsecond / 100000) * 100 + second_rem) * 1000
        diff_one = abs(current_microsecond - new_microsecond_one)
        diff_two = abs(current_microsecond - new_microsecond_one)
        if diff_one > diff_two:
            new_microsecond = new_microsecond_two
        else:
            new_microsecond = new_microsecond_one
        return date.replace(microsecond=new_microsecond)

    index_first_date = fixed_size_df.index[0]
    milli = index_first_date.microsecond / 1000
    first_remainder = int(milli % 100)
    second_remainder = int((milli + 50) % 100)

    updated_step_dates = set()
    for date in step_dates:
        updated_step_dates.add(update_date_microsecond(date, first_remainder, second_remainder))

    is_step_series = []
    count = 0
    for m in fixed_size_df.index:
        is_step = m.to_pydatetime() in updated_step_dates
        if is_step:
            count += 1
        is_step_series.append(is_step)

    fixed_size_df['isStep'] = is_step_series

    # Step 5
    def normalize(df):
        feature_count = df.shape[1] - 2
        values = df.iloc[:, 0:feature_count]
        phone_labels = df.loc[:, "labelPhone"].set_axis(range(df.shape[0]))
        is_step = df.loc[:, "isStep"].set_axis(range(df.shape[0]))
        normalizer = MinMaxScaler()
        normalized_values = normalizer.fit_transform(values)
        normalized_df = pd.DataFrame(
            data=normalized_values,
            columns=values.columns
        )
        normalized_df["labelPhone"] = phone_labels
        normalized_df["isStep"] = is_step
        return normalized_df

    normalized_df = normalize(fixed_size_df)

    # Step 6
    def count_number_of_steps(list):
        step_count = 0
        for item in list:
            if item:
                step_count += 1
        return step_count

    def convert_df_to_final_train_data(df):

        train_x, test_x = [], []
        train_x_lite, test_x_lite = [], []
        train_y_context, test_y_context = [], []
        train_y_step, test_y_step = [], []

        window_index_start = 0
        window_index_increasing_size = int(WINDOW_SIZE / 8)  # 2
        no_features = df.shape[1] - 2
        values = df.iloc[:, 0:no_features]
        context_labels = df.loc[:, "labelPhone"]
        step_found = df.loc[:, "isStep"]

        while window_index_start + WINDOW_SIZE < df.shape[0]:
            train_data = values[window_index_start:(window_index_start + WINDOW_SIZE)]
            train_data_lite = train_data[-2:]
            context_label = context_labels[window_index_start]
            step_count = count_number_of_steps(step_found[window_index_start:(window_index_start + WINDOW_SIZE)])
            # is_step_list = step_found[window_index_start:(window_index_start + WINDOW_SIZE)]

            if random() < 0.15:
                test_x_lite.append(train_data_lite)
                test_x.append(train_data)
                test_y_context.append(context_label)
                # test_y_step.append(is_step_list)
                test_y_step.append(step_count)
            else:
                train_x_lite.append(train_data_lite)
                train_x.append(train_data)
                train_y_context.append(context_label)
                # train_y_step.append(is_step_list)
                train_y_step.append(step_count)

            window_index_start += window_index_increasing_size

        return train_x, train_x_lite, train_y_step, train_y_context, test_x, test_x_lite, test_y_step, test_y_context

    return convert_df_to_final_train_data(normalized_df)


In [10]:
train_x, train_x_lite, train_y_step, train_y_context = [], [], [], []
test_x, test_x_lite, test_y_step, test_y_context = [], [], [], []

# mm, tt, pp, ds, os
inside_pant_pocket = ["tt1", "pp1", "ds4"] # 75%
inside_the_bag = ["tt5", "pp5", "ds3"]
swinging_in_hand = ["tt3", "pp3", "ds5"]
for code in swinging_in_hand:
    print("Handling data from code " + code)
    a, b, c, d, e, f, g, h = load_data_from_code(code)

    train_x = train_x + a
    train_x_lite = train_x_lite + b
    train_y_step = train_y_step + c
    train_y_context = train_y_context + d
    test_x = test_x + e
    test_x_lite = test_x_lite + f
    test_y_step = test_y_step + g
    test_y_context = test_y_context + h

train_x = np.array(train_x)
train_x_lite = np.array(train_x_lite)
train_y_step = np.array(train_y_step)
train_y_context = np.array(train_y_context)
test_x = np.array(test_x)
test_x_lite = np.array(test_x_lite)
test_y_step = np.array(test_y_step)
test_y_context = np.array(test_y_context)

print_line_divider()
print(train_x.shape, train_x_lite.shape, train_y_step.shape, train_y_context.shape)
print(test_x.shape, test_x_lite.shape, test_y_step.shape, test_y_context.shape)

Handling data from code tt3
Handling data from code pp3
Handling data from code ds5
****************************************************
(3082, 40, 9) (3082, 2, 9) (3082,) (3082,)
(517, 40, 9) (517, 2, 9) (517,) (517,)


## Training and testing

#### Simple binary classifier using CNN

In [28]:
from keras import layers, models, Sequential
from keras import backend as K


def build_shallow_cnn(input_shape):
    model_2d = Sequential()
    model_2d.add(
        layers.Conv1D(
            filters=12,
            kernel_size=3,
            strides=1,
            padding="same",
            activation='relu',
            input_shape=input_shape)
    )
    model_2d.add(layers.Dropout(0.1))
    model_2d.add(layers.MaxPooling1D(pool_size=2, strides=2, padding="valid"))
    model_2d.add(layers.Flatten())
    model_2d.add(layers.Dense(50, activation='relu'))
    model_2d.add(layers.Dropout(0.4))
    model_2d.add(layers.Dense(1, activation='relu'))
    return model_2d


def build_cnn_step_model(input_shape):
    input_layer = layers.Input(input_shape)
    conv1 = layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.ReLU()(conv1)
    conv2 = layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.ReLU()(conv2)
    conv3 = layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.ReLU()(conv3)
    gap = layers.GlobalAveragePooling1D()(conv2)
    dense = layers.Dense(128, activation='relu')(gap)
    output_layer = layers.Dense(1, activation="relu")(dense)

    return models.Model(inputs=input_layer, outputs=output_layer)


model = build_cnn_step_model((40, 9))
# model = build_shallow_cnn((40,9))
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 40, 9)]           0         
                                                                 
 conv1d_7 (Conv1D)           (None, 40, 64)            1792      
                                                                 
 batch_normalization_6 (Batc  (None, 40, 64)           256       
 hNormalization)                                                 
                                                                 
 re_lu_6 (ReLU)              (None, 40, 64)            0         
                                                                 
 conv1d_8 (Conv1D)           (None, 40, 64)            12352     
                                                                 
 batch_normalization_7 (Batc  (None, 40, 64)           256       
 hNormalization)                                           

In [ ]:
from keras import callbacks
import tensorflow as tf

SAVED_BEST_MODEL = "model/best_step_model.h5"
callback_list = [
    callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=50, min_lr=0.0001),
    # callbacks.EarlyStopping(monitor="val_loss", patience=500, verbose=1),
    callbacks.ModelCheckpoint(SAVED_BEST_MODEL, save_best_only=True, monitor="val_loss")
]
epochs = 300
batch_size = 32
validation_split = 15 / 85
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.005,
    name="Adam",
)


model.compile(
    optimizer=optimizer,
    loss='mse',
    # metrics=['accuracy']
)
history = model.fit(
    train_x,
    train_y_step,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callback_list,
    validation_split=validation_split,
    verbose=1,
    shuffle=True
)
model = models.load_model(SAVED_BEST_MODEL)

Epoch 1/300
80/80 [==============================] - 1s 11ms/step - loss: 1.0760 - val_loss: 2.2541 - lr: 0.0050
Epoch 2/300
80/80 [==============================] - 1s 9ms/step - loss: 0.6063 - val_loss: 2.3207 - lr: 0.0050
Epoch 3/300
80/80 [==============================] - 1s 9ms/step - loss: 0.5737 - val_loss: 1.5519 - lr: 0.0050
Epoch 4/300
80/80 [==============================] - 1s 10ms/step - loss: 0.5393 - val_loss: 1.8999 - lr: 0.0050
Epoch 5/300
80/80 [==============================] - 1s 10ms/step - loss: 0.5478 - val_loss: 1.0404 - lr: 0.0050
Epoch 6/300
80/80 [==============================] - 1s 8ms/step - loss: 0.4861 - val_loss: 1.1340 - lr: 0.0050
Epoch 7/300
80/80 [==============================] - 1s 9ms/step - loss: 0.4845 - val_loss: 5.7941 - lr: 0.0050
Epoch 8/300
80/80 [==============================] - 1s 10ms/step - loss: 0.4318 - val_loss: 1.9868 - lr: 0.0050
Epoch 9/300
80/80 [==============================] - 1s 9ms/step - loss: 0.4468 - val_loss: 2.2246 -

In [ ]:
import matplotlib.pyplot as plt

# Plot the result
metric = "loss"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("CNN:" + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()

In [ ]:
def round_pred(pred):
    round_list = []
    for i in pred:
        round_list.append(round(i[0]))
    return round_list

def print_correct_count(pred, actual):
    old_pred = pred
    pred = round_pred(pred)
    correct_count = 0
    for i in range(len(actual)):
        if actual[i] == pred[i]:
            print("Correct: ", pred[i])
            correct_count += 1
        else:
            print("Incorrect: index - ", i, " | base - ", old_pred[i][0] ," | predict - ", pred[i], " | actual - ", actual[i])
    print("Correct: ", correct_count, "/", len(actual))


y_pred = model.predict(test_x)
print_correct_count(y_pred, test_y_step)

print_line_divider()